In [2]:
import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [3]:
import itertools

def powerset(iterable):
    s = list(iterable)
    return itertools.chain.from_iterable(
            itertools.combinations(s, r) for r in range(len(s)+1))

In [4]:
import json
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
import modeling.utils

In [5]:
# For scikit learn metrics.
precision_recall_average = 'macro'

In [6]:
# Best so far, but imbalanced.
model_dir = 'models/keras/preposition/convnet/20a7a6b088ee11e5b2b374d435ed6f3a/'

# Balanced.
# model_dir = 'models/keras/preposition/convnet/balanced/'

# Load the test set for evaluation.
data_file = 'data/preposition/prepositions-all-new-test.h5'

In [7]:
model, model_cfg = modeling.utils.load_model(model_dir, load_weights=True)

Loading weights (build_model)
Loading weights


Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled)


In [8]:
model_data = modeling.utils.load_all_model_data(data_file, model_cfg)

[(999552, 5), (999552, 52)]


In [9]:
# Load target data or metadata (e.g. mapping between numeric target variable and preposition).
target_data_file = 'data/preposition/prepositions-all-new-target-data.json'
target_data = json.load(open(target_data_file))

In [ ]:
def compute_n_unknown_words():
    n_unknown_words = np.zeros_like(model_data.len)
    for i in np.arange(0, len(model_data.len)):
        n_unknown_words[i] = len(np.where(model_data.data[i, 0:model_data.len[i]] == 0)[0])
    return n_unknown_words

Sensitivity analysis of effect of position of unknown words in window around preposition
=======
1. Take all examples in which the window around the preposition contains no unknown words.
2. For each set in the powerset of positions in the window (excluding the center, where the preposition occurs):
  1. Set the words in that position to be unknown (i.e. assign 0 to that position) for all examples.
  2. Run the examples through the model.
3. Evaluate the model's performance.

In [ ]:
def sensitivity_analysis(n=50000):
    n_unknown_words = compute_n_unknown_words()

    print('# of examples ' + str(len(model_data.data)))
    print('# of examples with no unknown words ' + str((n_unknown_words==0).sum()))
    
    error_detection_targets = np.ones_like(model_data.current_word_code)
    evens = np.arange(0, len(model_data.target), 2)
    error_detection_targets[evens] = 0

    no_unknown_words_data = model_data.data[n_unknown_words == 0]
    no_unknown_words_correction_targets = model_data.target[n_unknown_words == 0]
    no_unknown_words_detection_targets = error_detection_targets[n_unknown_words == 0]

    window_size = 5
    center = 2

    assert len(np.where(model_data.data[:, center] == 0)[0]) == 0

    indices_in_window = [center-2, center-1, center+1, center+2]

    masks = [mask for mask in powerset(indices_in_window)]

    correction_results = {}
    
    results_df = None

    for mask in masks:
        data = no_unknown_words_data.copy()[0:n]
        mask = np.array(mask, dtype=int)

        data[:, mask] = 0

        for i in np.arange(len(data)):
            data[i, mask + model_data.position[i] + 3] = 0

        no_unknown_words_correction_preds = model.predict_classes(data, verbose=0)

        unknowns_str = ['_'] * (len(indices_in_window) + 1)
        for x in mask:
            unknowns_str[x] = "?"
        unknowns_str[center] = "P"

        # Error correction
        p, r, f, _ = precision_recall_fscore_support(
                no_unknown_words_correction_targets[0:n],
                no_unknown_words_correction_preds,
                average=precision_recall_average)
        
        row = pd.DataFrame({
                "pos-2": [unknowns_str[0]],
                "pos-1": [unknowns_str[1]],
                "pos-0": [unknowns_str[2]],
                "pos+1": [unknowns_str[3]],
                "pos+2": [unknowns_str[4]],
                "precision": [p],
                "recall": [r],
                "f1": [f],
                "n": [n]
                    })
        if results_df is None:
            results_df = row
        else:
            results_df = pd.concat([results_df, row])

    results_df = results_df[["pos-2", "pos-1", "pos-0", "pos+1", "pos+2", "precision", "recall", "f1", "n"]]
    print(results_df.to_latex(index=False, float_format=lambda f: '%.02f' % f))
      
sensitivity_analysis(n=10000)